In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt

parameters = {
    'axes.labelsize':  14,
    'figure.titlesize': 16,
    'xtick.labelsize': 12,
    'ytick.labelsize': 12,
    'axes.titlesize': 16,
    'legend.fontsize': 12,
    'legend.title_fontsize': 14
}
plt.rcParams.update(parameters)

from sklearn.metrics import mean_absolute_error

from sklearn.linear_model import LinearRegression, Lasso, Ridge
from sklearn.preprocessing import PolynomialFeatures

Попробуйте создать искусственный датасет с лишними столбцами. Целевую метку, при правильной обработке данных, сформировать таким образом, чтобы без затруднений её смогла описать линейная модель. Данное задание не имеет какого-то “правильного” решения. Тем не менее, стоит представить те ситуации, которые могли бы быть в реальных данных.

### Создание семпла

Сделаем искусственный семпл описывающий размер заработной платы в зависимости от количества лет работы сотрудника, его уровня IQ пола, возраста.

In [2]:
# Создаём сэмпл
n_samples = 1000

# опыт работы в годах
experience_years = np.random.choice(15, n_samples)
# возраст начала работы
work_start_age = np.random.choice(30, n_samples) + 18
# возраст сотрудника
age = work_start_age + experience_years
# уровень IQ
IQ = np.random.choice(80, n_samples) + 70
# пол
sex = np.random.choice(1, n_samples)
# вес
hight = np.random.choice(70, n_samples) + 50

Искусственно сгенерируем велчину зарплаты. Она зависит от опыта и быстро растёт в первые годы рабты, замедляя свой рост в последующие. Эту закономерность отразим, взяв квадратный корень от опыта работы в годах. Также зарплата выше у сотрудников с более высоким IQ. От пола и роста у нас зарплата не зависит.

In [3]:
salary = 250*np.sqrt(experience_years) + IQ*500 + 20_000

data = pd.DataFrame({
    'experience': experience_years,
    'age': age,
    'IQ': IQ,
    'sex': sex,
    'hight': hight,
    'salary': salary
})
data.head(5)

,experience,age,IQ,sex,hight,salary
0,5,29,95,0,80,68059.016994
1,9,40,104,0,101,72750.000000
2,8,42,98,0,89,69707.106781
3,4,29,94,0,54,67500.000000
4,5,25,79,0,106,60059.016994


In [4]:
data.describe()

,experience,age,IQ,sex,hight,salary
count,1000.000000,1000.000000,1000.000000,1000.0,1000.000000,1000.000000
mean,7.045000,39.241000,110.233000,0.0,86.056000,75726.951890
std,4.337256,9.770075,23.386655,0.0,20.178256,11676.453384
min,0.000000,18.000000,70.000000,0.0,50.000000,55000.000000
25%,3.000000,32.000000,89.000000,0.0,70.000000,64935.414347
50%,7.000000,39.000000,110.000000,0.0,87.000000,75770.284708
75%,11.000000,47.000000,131.000000,0.0,104.000000,86260.142354
max,14.000000,61.000000,149.000000,0.0,119.000000,95435.414347


Построим линейную модель на основе всех данных

In [5]:
X = data[['experience', 'age', 'IQ', 'sex', 'hight']]
y = data['salary']
reg = LinearRegression().fit(X, y)
print('Weights: {}'.format(reg.coef_))

pred_values = reg.predict(X)
print('Bias: {}'.format(pred_values.mean() - y.mean()))

print('Error: {}'.format(mean_absolute_error(pred_values, y)))

Weights: [ 5.76573013e+01 -1.40458610e-02  4.99922567e+02  0.00000000e+00
 -8.75904934e-03]
Bias: 1.4551915228366852e-11
Error: 53.80878356552398


Модель смогла понять, что ни возраст, ни пол, ни рост не имеют определяющего значения и приписала им очень малые веса.

In [6]:
X = data[['experience', 'IQ']]
y = data['salary']
reg = LinearRegression().fit(X, y)
print('Weights: {}'.format(reg.coef_))

pred_values = reg.predict(X)
print('Bias: {}'.format(pred_values.mean() - y.mean()))

print('Error: {}'.format(mean_absolute_error(pred_values, y)))

Weights: [ 57.64355325 499.92302743]
Bias: 0.0
Error: 53.81929072456729


Если построить модель без лишних переменных, то ошибка окажется практически такой же, как и с ними.

In [7]:
poly = PolynomialFeatures(3)

In [8]:
X = data[['experience', 'age', 'IQ', 'sex', 'hight']]
X = poly.fit_transform(X)

y = data['salary']
reg = LinearRegression().fit(X, y)
print('Weights: {}'.format(reg.coef_))

pred_values = reg.predict(X)
print('Bias: {}'.format(pred_values.mean() - y.mean()))

print('Error: {}'.format(mean_absolute_error(pred_values, y)))

Weights: [ 1.27762499e-08  1.64139087e+02  6.40489766e-01  4.99791743e+02
  1.47093342e-09 -5.06181783e+00 -1.37385675e+01 -1.02454141e-01
  1.93990169e-01  9.12135437e-11 -2.33550927e-01 -2.22246430e-02
 -1.41546306e-02  1.30700284e-10  2.47169219e-02 -5.33660900e-03
  1.01695430e-10 -3.37926513e-04 -6.18238793e-12 -1.65467640e-11
  6.59533256e-02  4.80991396e-01  7.51738402e-03 -3.92076714e-03
  4.13180601e-12  2.43653727e-03 -8.46139405e-04 -9.46680785e-04
 -1.49213975e-13  1.50445817e-03 -3.56226863e-04 -5.68434189e-14
 -2.83011357e-04 -5.68434189e-14  2.84217094e-14  9.99739974e-04
  2.15466707e-04  1.65913145e-04  0.00000000e+00 -1.20299059e-04
  1.30184937e-04  0.00000000e+00 -1.68408424e-04  0.00000000e+00
  0.00000000e+00 -6.66642893e-05 -4.36273712e-06  0.00000000e+00
  7.50328944e-05  0.00000000e+00  0.00000000e+00 -3.14913138e-05
  0.00000000e+00  0.00000000e+00  0.00000000e+00 -2.65541915e-04]
Bias: -1.4551915228366852e-11
Error: 17.493057261100816


Использование полиномиальных фич позволяет значительно снизить ошибку модели не внося заметного отклонения. Это происходит потому, что сумма степеней от опыта начинают аппроксимировать квадратный корень из опыта.

In [9]:
poly = PolynomialFeatures(3)

In [10]:
X = data[['experience', 'IQ']]
X['experience'] = np.sqrt(X['experience'])

y = data['salary']
reg = LinearRegression().fit(X, y)
print('Weights: {}'.format(reg.coef_))

pred_values = reg.predict(X)
print('Bias: {}'.format(pred_values.mean() - y.mean()))

print('Error: {}'.format(mean_absolute_error(pred_values, y)))

Weights: [250. 500.]
Bias: 0.0
Error: 9.029463399201632e-12


<ipython-input-10-32abf1299554>:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  X['experience'] = np.sqrt(X['experience'])


Замена опыта на квадратный корень из него позволяет уменьшить ошибку почти до нуля.